In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rossmann-store-sales/sample_submission.csv
/kaggle/input/rossmann-store-sales/store.csv
/kaggle/input/rossmann-store-sales/train.csv
/kaggle/input/rossmann-store-sales/test.csv


In [2]:
store = pd.read_csv("../input/rossmann-store-sales/store.csv")
train = pd.read_csv('../input/rossmann-store-sales/train.csv',parse_dates = [2])
test = pd.read_csv('../input/rossmann-store-sales/test.csv',parse_dates = [3])

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,a,a,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,c,c,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,a,c,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,a,c,870.0,NaN,NaN,0,NaN,NaN,NaN


In [4]:
# check store nan rows
store.isnull().sum()

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [5]:
col = store.columns.values

In [6]:
store.PromoInterval.value_counts()

Jan,Apr,Jul,Oct     335
Feb,May,Aug,Nov     130
Mar,Jun,Sept,Dec    106
Name: PromoInterval, dtype: int64

In [7]:
# check store nan rows
test.isnull().sum()

Id                0
Store             0
DayOfWeek         0
Date              0
Open             11
Promo             0
StateHoliday      0
SchoolHoliday     0
dtype: int64

In [8]:
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
# mappings2 = {'Jan,Apr,Jul,Oct':0,'Feb,May,Aug,Nov':1,'Mar,Jun,Sept,Dec':2}
store.StoreType.replace(mappings, inplace=True)
store.Assortment.replace(mappings, inplace=True)
# store.PromoInterval.replace(mappings2, inplace=True)

In [9]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,1,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,1,1,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,3,3,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,1,1,29910.0,4.0,2015.0,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1110,1111,1,1,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,3,3,1880.0,4.0,2006.0,0,NaN,NaN,NaN
1112,1113,1,3,9260.0,NaN,NaN,0,NaN,NaN,NaN
1113,1114,1,3,870.0,NaN,NaN,0,NaN,NaN,NaN


In [10]:
# We will not impute as it is already complete, we just set it zero as these variables do not affect prediction
# from sklearn.impute import KNNImputer
# KNN_I = KNNImputer(n_neighbors=5)
# x = KNN_I.fit_transform(store[['Store','StoreType','Assortment','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2']])
# x
store.fillna(0,inplace = True)

In [11]:
# store_I = pd.DataFrame(x,columns=['Store','StoreType','Assortment','CompetitionDistance','CompetitionOpenSinceMonth','CompetitionOpenSinceYear','Promo2'])

In [12]:
# store = pd.merge(store[['Store','Promo2SinceWeek','Promo2SinceYear','PromoInterval']],store_I,on='Store')

In [13]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,1,1,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,3,3,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,1,1,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
1110,1111,1,1,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,3,3,1880.0,4.0,2006.0,0,0.0,0.0,0
1112,1113,1,3,9260.0,0.0,0.0,0,0.0,0.0,0
1113,1114,1,3,870.0,0.0,0.0,0,0.0,0.0,0


In [14]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1
...,...,...,...,...,...,...,...,...,...
1017204,1111,2,2013-01-01,0,0,0,0,a,1
1017205,1112,2,2013-01-01,0,0,0,0,a,1
1017206,1113,2,2013-01-01,0,0,0,0,a,1
1017207,1114,2,2013-01-01,0,0,0,0,a,1


In [15]:
store

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0
1,2,1,1,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,3,3,620.0,9.0,2009.0,0,0.0,0.0,0
4,5,1,1,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...
1110,1111,1,1,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1112,3,3,1880.0,4.0,2006.0,0,0.0,0.0,0
1112,1113,1,3,9260.0,0.0,0.0,0,0.0,0.0,0
1113,1114,1,3,870.0,0.0,0.0,0,0.0,0.0,0


In [16]:
store.reset_index()

,index,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0
1,1,2,1,1,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,2,3,1,1,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,3,4,3,3,620.0,9.0,2009.0,0,0.0,0.0,0
4,4,5,1,1,29910.0,4.0,2015.0,0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
1110,1110,1111,1,1,1900.0,6.0,2014.0,1,31.0,2013.0,"Jan,Apr,Jul,Oct"
1111,1111,1112,3,3,1880.0,4.0,2006.0,0,0.0,0.0,0
1112,1112,1113,1,3,9260.0,0.0,0.0,0,0.0,0.0,0
1113,1113,1114,1,3,870.0,0.0,0.0,0,0.0,0.0,0


In [17]:
store['Store']

0          1
1          2
2          3
3          4
4          5
        ... 
1110    1111
1111    1112
1112    1113
1113    1114
1114    1115
Name: Store, Length: 1115, dtype: int64

In [18]:
# merge data with store 
train = train.merge(store,on='Store')
test = test.merge(store, on='Store')

In [19]:
train = train.sort_values(['Date'],ascending = False)
# train_total = train.copy()

# split_index = 6*7*1115
# valid = train[:split_index] 
# train = train[split_index:]

In [20]:
train

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0
679364,747,5,2015-07-31,10708,826,1,1,0,1,3,3,45740.0,8.0,2008.0,0,0.0,0.0,0
702362,772,5,2015-07-31,5224,635,1,1,0,1,4,3,1850.0,0.0,0.0,0,0.0,0.0,0
683890,752,5,2015-07-31,7763,565,1,1,0,1,1,1,970.0,3.0,2013.0,1,31.0,2013.0,"Feb,May,Aug,Nov"
17714,20,5,2015-07-31,9593,974,1,1,0,0,4,1,2340.0,5.0,2009.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236455,260,2,2013-01-01,0,0,0,0,a,1,1,1,540.0,10.0,2011.0,0,0.0,0.0,0
907664,995,2,2013-01-01,0,0,0,0,a,1,4,1,6560.0,12.0,2013.0,0,0.0,0.0,0
237397,261,2,2013-01-01,0,0,0,0,a,1,4,3,15340.0,4.0,2015.0,1,39.0,2009.0,"Jan,Apr,Jul,Oct"
238339,262,2,2013-01-01,17267,2875,1,0,a,1,2,1,1180.0,5.0,2013.0,0,0.0,0.0,0


In [21]:
#making fair assumption that when the store is not open sales sales should be zero. Discarding training on non zero values
train  = train[(train.Open != 0)&(train.Sales != 0) ]
test = test[test.Open != 0]

In [22]:
train.StateHoliday.replace(mappings, inplace=True)
test.StateHoliday.replace(mappings, inplace=True)


/opt/conda/lib/python3.7/site-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


In [23]:
#Creating features for Month, Year, Day, WeekofYear to make the algorithm learn on the time series
train['Month'] = train.Date.dt.month
train['Year'] = train.Date.dt.year
train['Day'] = train.Date.dt.day
train['WeekofYear'] = train.Date.dt.weekofyear
test['Month'] = test.Date.dt.month
test['Year'] = test.Date.dt.year
test['Day'] = test.Date.dt.day
test['WeekofYear'] = test.Date.dt.weekofyear

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [24]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,...,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekofYear
0,1,5,2015-07-31,5263,555,1,1,0,1,3,...,9.0,2008.0,0,0.0,0.0,0,7,2015,31,31
679364,747,5,2015-07-31,10708,826,1,1,0,1,3,...,8.0,2008.0,0,0.0,0.0,0,7,2015,31,31
702362,772,5,2015-07-31,5224,635,1,1,0,1,4,...,0.0,0.0,0,0.0,0.0,0,7,2015,31,31
683890,752,5,2015-07-31,7763,565,1,1,0,1,1,...,3.0,2013.0,1,31.0,2013.0,"Feb,May,Aug,Nov",7,2015,31,31
17714,20,5,2015-07-31,9593,974,1,1,0,0,4,...,5.0,2009.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct",7,2015,31,31


In [25]:
train.isnull().sum()

Store                        0
DayOfWeek                    0
Date                         0
Sales                        0
Customers                    0
Open                         0
Promo                        0
StateHoliday                 0
SchoolHoliday                0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
Month                        0
Year                         0
Day                          0
WeekofYear                   0
dtype: int64

In [26]:
def checkrow(row):
    var = 1 if isinstance(row['PromoInterval'],str) and row['month_str'] in row['PromoInterval'] else 0
    return var

In [27]:
# Indicate whether the month is in promo interval
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', \
             7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
train['month_str'] = train.Month.map(month2str)
test['month_str'] = test.Month.map(month2str)
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [28]:
train['IsInPromoInterval'] = train.apply(lambda row: checkrow(row),axis=1)
test['IsInPromoInterval'] = test.apply(lambda row: checkrow(row),axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
pd.set_option('max_columns', 30)
train[train['month_str'] == 'Nov'].head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekofYear,month_str,IsInPromoInterval
511961,562,7,2014-11-30,24801,3507,1,0,0,0,2,3,1210.0,0.0,0.0,0,0.0,0.0,0,11,2014,30,48,Nov,0
237641,262,7,2014-11-30,36417,4816,1,0,0,0,2,1,1180.0,5.0,2013.0,0,0.0,0.0,0,11,2014,30,48,Nov,0
615301,676,7,2014-11-30,14237,2187,1,0,0,0,2,2,1410.0,9.0,2008.0,0,0.0,0.0,0,11,2014,30,48,Nov,0
320787,353,7,2014-11-30,9531,1836,1,0,0,0,2,2,900.0,0.0,0.0,1,14.0,2013.0,"Feb,May,Aug,Nov",11,2014,30,48,Nov,1
76979,85,7,2014-11-30,15386,1833,1,0,0,0,2,1,1870.0,10.0,2011.0,0,0.0,0.0,0,11,2014,30,48,Nov,0


In [30]:

train['CompetitionOpen'] = 12*(train.Year - train.CompetitionOpenSinceYear)+(train.Month-train.CompetitionOpenSinceMonth) #if train.CompetitionOpenSinceYear>0 else 0

train['PromoOpen'] = 12*(train.Year - train.Promo2SinceYear)+(train.WeekofYear-train.Promo2SinceWeek) #if train.Promo2SinceYear >0 else 0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
def checkrow2(row):
    var = 0 if  row['CompetitionOpenSinceYear']== 0  else  row['CompetitionOpen']
    return var

In [32]:
def checkrow3(row):
    var = 0 if  row['Promo2']== 0  else  row['PromoOpen']
    return var

In [33]:
train['CompetitionOpen'] = train.apply(lambda row:checkrow2(row),axis=1)
train['PromoOpen'] = train.apply(lambda row:checkrow3(row),axis=1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [34]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekofYear,month_str,IsInPromoInterval,CompetitionOpen,PromoOpen
0,1,5,2015-07-31,5263,555,1,1,0,1,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,7,2015,31,31,Jul,0,82.0,0.0
679364,747,5,2015-07-31,10708,826,1,1,0,1,3,3,45740.0,8.0,2008.0,0,0.0,0.0,0,7,2015,31,31,Jul,0,83.0,0.0
702362,772,5,2015-07-31,5224,635,1,1,0,1,4,3,1850.0,0.0,0.0,0,0.0,0.0,0,7,2015,31,31,Jul,0,0.0,0.0
683890,752,5,2015-07-31,7763,565,1,1,0,1,1,1,970.0,3.0,2013.0,1,31.0,2013.0,"Feb,May,Aug,Nov",7,2015,31,31,Jul,0,28.0,24.0
17714,20,5,2015-07-31,9593,974,1,1,0,0,4,1,2340.0,5.0,2009.0,1,40.0,2014.0,"Jan,Apr,Jul,Oct",7,2015,31,31,Jul,1,74.0,3.0


In [35]:
test['CompetitionOpen'] = 12*(test.Year - test.CompetitionOpenSinceYear)+(test.Month-test.CompetitionOpenSinceMonth) #if test.CompetitionOpenSinceYear>0 else 0

test['PromoOpen'] = 12*(test.Year - test.Promo2SinceYear)+(test.WeekofYear-test.Promo2SinceWeek) #if test.Promo2SinceYear >0 else 0
test['CompetitionOpen'] = test.apply(lambda row:checkrow2(row),axis=1)
test['PromoOpen'] = test.apply(lambda row:checkrow3(row),axis=1)

In [36]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,Month,Year,Day,WeekofYear,month_str,IsInPromoInterval,CompetitionOpen,PromoOpen
0,1,1,4,2015-09-17,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,9,2015,17,38,Sept,0,84.0,0.0
1,857,1,3,2015-09-16,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,9,2015,16,38,Sept,0,84.0,0.0
2,1713,1,2,2015-09-15,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,9,2015,15,38,Sept,0,84.0,0.0
3,2569,1,1,2015-09-14,1.0,1,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,9,2015,14,38,Sept,0,84.0,0.0
5,4281,1,6,2015-09-12,1.0,0,0,0,3,1,1270.0,9.0,2008.0,0,0.0,0.0,0,9,2015,12,37,Sept,0,84.0,0.0


In [37]:
test.isnull().sum()

Id                            0
Store                         0
DayOfWeek                     0
Date                          0
Open                         11
Promo                         0
StateHoliday                  0
SchoolHoliday                 0
StoreType                     0
Assortment                    0
CompetitionDistance           0
CompetitionOpenSinceMonth     0
CompetitionOpenSinceYear      0
Promo2                        0
Promo2SinceWeek               0
Promo2SinceYear               0
PromoInterval                 0
Month                         0
Year                          0
Day                           0
WeekofYear                    0
month_str                     0
IsInPromoInterval             0
CompetitionOpen               0
PromoOpen                     0
dtype: int64

In [38]:
test.fillna(1,inplace = True)

In [39]:
train[['Store','DayOfWeek','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo2','Month','Year','Day','WeekofYear','IsInPromoInterval','CompetitionOpen','PromoOpen']]

,Store,DayOfWeek,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,Month,Year,Day,WeekofYear,IsInPromoInterval,CompetitionOpen,PromoOpen
0,1,5,1,0,1,3,1,1270.0,0,7,2015,31,31,0,82.0,0.0
679364,747,5,1,0,1,3,3,45740.0,0,7,2015,31,31,0,83.0,0.0
702362,772,5,1,0,1,4,3,1850.0,0,7,2015,31,31,0,0.0,0.0
683890,752,5,1,0,1,1,1,970.0,1,7,2015,31,31,0,28.0,24.0
17714,20,5,1,0,0,4,1,2340.0,1,7,2015,31,31,1,74.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
864127,948,2,0,1,1,2,2,1430.0,0,1,2013,1,1,0,0.0,0.0
321485,353,2,0,1,1,2,2,900.0,1,1,2013,1,1,0,0.0,-13.0
249459,274,2,0,1,1,2,2,3640.0,1,1,2013,1,1,1,0.0,-9.0
235513,259,2,0,1,1,2,2,210.0,0,1,2013,1,1,0,0.0,0.0


In [40]:
import xgboost as xgb
# Instantiate an XGBoost object with hyperparameters
dtrain = train[['Store','DayOfWeek','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo2','Month','Year','Day','WeekofYear','IsInPromoInterval','CompetitionOpen','PromoOpen','CompetitionOpenSinceMonth','Promo2SinceWeek']]
# ,train['Sales']
dtest = test[['Store','DayOfWeek','Promo','StateHoliday','SchoolHoliday','StoreType','Assortment','CompetitionDistance','Promo2','Month','Year','Day','WeekofYear','IsInPromoInterval','CompetitionOpen','PromoOpen','CompetitionOpenSinceMonth','Promo2SinceWeek']]
# specify parameters via map
params = {"objective": ["reg:linear"], # for linear regression
          "booster" : ["gbtree"],   # use tree based models 
          "eta": [0.03,0.5,0.1],   # learning rate
          "max_depth": [3,5,10],    # maximum depth of a tree
          "subsample": [0.9],    # Subsample ratio of the training instances
          "colsample_bytree": [0.7],   # Subsample ratio of columns when constructing each tree
          "silent": [1],   # silent mode
          "seed": [10]   # Random number seed
          }


In [41]:
from sklearn.model_selection import GridSearchCV
xgb_grid = GridSearchCV(xgb.XGBRegressor(), params, n_jobs=5,
                   verbose=2, refit=True)

In [42]:
xgb_grid.fit(dtrain,
         train['Sales'])
# make prediction

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed: 30.0min


[CV] booster=gbtree, colsample_bytree=0.7, eta=0.03, max_depth=3, objective=reg:linear, seed=10, silent=1, subsample=0.9 
[09:13:46] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[09:13:46] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[CV]  booster=gbtree, colsample_bytree=0.7, eta=0.03, max_depth=3, objective=reg:linear, seed=10, silent=1, subsample=0.9, total= 2.1min
[CV] booster=gbtree, colsample_bytree=0.7, eta=0.03, max_depth=5, objective=reg:linear, seed=10, silent=1, subsample=0.9 
[09:15:54] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[09:15:54] WARNING: ../src/learner.cc:573: 
Parameter

[Parallel(n_jobs=5)]: Done  45 out of  45 | elapsed: 43.0min finished


[09:56:41] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[09:56:41] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=None,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
      

In [43]:

xgb_grid.best_score_

0.8406273276021718

In [44]:
xgb_grid.best_params_

{'booster': 'gbtree',
 'colsample_bytree': 0.7,
 'eta': 0.5,
 'max_depth': 10,
 'objective': 'reg:linear',
 'seed': 10,
 'silent': 1,
 'subsample': 0.9}

In [45]:

parameters = {"objective": "reg:linear", # for linear regression
          "booster" : "gbtree",   # use tree based models 
          "eta": 0.5,   # learning rate
          "max_depth": 10,    # maximum depth of a tree
          "subsample": 0.9,    # Subsample ratio of the training instances
          "colsample_bytree": 0.7,   # Subsample ratio of columns when constructing each tree
          "silent": 1,   # silent mode
          "seed": 10   # Random number seed
          }
xgbR = xgb.XGBRegressor(objective= "reg:linear", # for linear regression
          booster= "gbtree",   # use tree based models 
          eta= 0.5,   # learning rate
          max_depth= 10,    # maximum depth of a tree
          subsample= 0.9,    # Subsample ratio of the training instances
          colsample_bytree= 0.7,   # Subsample ratio of columns when constructing each tree
          silent= 1,   # silent mode
          seed= 10)
xgbR.fit(
    dtrain, 
    train['Sales'], 
    eval_metric="rmse")

[09:57:52] WARNING: ../src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[09:57:52] WARNING: ../src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.7, eta=0.5, gamma=0,
             gpu_id=-1, importance_type='gain', interaction_constraints='',
             learning_rate=0.5, max_delta_step=0, max_depth=10,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1,
             objective='reg:linear', random_state=10, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=10, silent=1, subsample=0.9,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [46]:
preds = xgb_grid.predict(dtest)

In [47]:
preds

array([4312.808 , 4622.0396, 4968.1733, ..., 6937.8745, 9595.771 ,
       5805.9272], dtype=float32)

In [48]:
preds.shape

(35104,)

In [49]:
df_test = pd.read_csv('../input/rossmann-store-sales/test.csv')
#Concatenating values where test stores were closed
df_test = df_test[df_test.Open == 0]
df_test['Sales'] = 0
df_test.shape

(5984, 9)

In [50]:
sub = pd.DataFrame({'Id':test['Id'],'Sales':preds})
sub1 = pd.concat([sub,df_test[['Id','Sales']]]).sort_values(['Id'],ascending = True)
sub1.to_csv("submission_ross3.csv", index=False)
sub1.shape

(41088, 2)

In [51]:
print(model)

NameError: name 'model' is not defined

In [ ]:
import pickle

with open('model_pickle.pkl','wb') as f: 
    pickle.dump(model,f) 